In [11]:
from langgraph.graph import StateGraph, START, END
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from typing import TypedDict, Annotated
from pydantic import Field, BaseModel
import operator
from dotenv import load_dotenv
import os

load_dotenv()

# Get API key from environment variable
hf_api_key = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [12]:
# Schema
class EvaluationSchema(BaseModel):

    feedback: str = Field(description="Detailed feedback for the essay")
    score: int = Field(description="Score out of 10", ge=0, le=10)


In [13]:
# Convert to JSON schema (Pydantic v2)
schema = EvaluationSchema.model_json_schema()

In [ ]:
# Structured Output Model
llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen3-Next-80B-A3B-Instruct",
    # repo_id="tiiuae/falcon-7b-instruct",
    task="task-generation",
    # huggingfacehub_api_token=hf_api_key
    huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_API_TOKEN")
)

model = ChatHuggingFace(llm=llm)

# import os
# from dotenv import load_dotenv
# from langchain_google_genai import ChatGoogleGenerativeAI

# load_dotenv()

# # Add to .env: GOOGLE_API_KEY=your-gemini-key
# model = ChatGoogleGenerativeAI(
#     model="gemini-1.5-flash",
#     google_api_key=os.getenv("GOOGLE_API_KEY")
# )

# structured_model = model.with_structured_output(schema)


In [15]:
structured_model = model.with_structured_output(schema)

In [16]:
essay = """
# 🌟 AI and Its Future in India

## 📌 Current Landscape

* **Rapid Growth**: India is among the **top 5 countries** in AI research publications.
* **Government Push**: Initiatives like **Digital India, National AI Strategy (NITI Aayog), and IndiaAI Mission** aim to build AI as a core pillar of development.
* **Startups & Industry**: Over **5,000+ AI startups** in India, focusing on healthtech, fintech, edtech, and agritech.
* **Skilled Workforce**: India produces the **largest pool of STEM graduates** annually, making it a hub for AI talent.

---

## 🔑 Key Application Areas

1. **Healthcare 🏥**

   * AI in medical imaging, disease prediction, and telemedicine.
   * Affordable AI-driven solutions for rural healthcare.

2. **Agriculture 🌾**

   * Precision farming with drones & AI sensors.
   * Predictive analytics for weather & crop yield.
   * Reducing supply chain inefficiencies.

3. **Education 🎓**

   * Personalized learning platforms with AI tutors.
   * Local-language learning tools powered by NLP.
   * Bridging urban–rural education gaps.

4. **Finance & Banking 💰**

   * Fraud detection & risk management.
   * AI-driven credit scoring for underserved communities.
   * Chatbots for customer support in regional languages.

5. **Governance & Smart Cities 🏙️**

   * Traffic management with AI vision systems.
   * Digital governance and citizen services.
   * Disaster response and resource optimization.

---

## 🚀 Opportunities Ahead

* **Demographic Advantage**: Young, tech-savvy population ready to adopt AI.
* **Data Wealth**: India generates massive datasets — essential for training AI systems.
* **Local Language AI**: A huge opportunity lies in **developing AI for Indian languages** (22 official + 100+ spoken).
* **Global AI Hub Potential**: India can become an outsourcing + innovation hub for AI solutions like it did for IT.

---

## ⚠️ Challenges to Overcome

* **Data Privacy & Security**: Need stronger AI governance and data protection laws.
* **Skill Gap**: Despite talent, advanced AI expertise (like deep learning, AGI research) is limited.
* **Infrastructure**: Limited GPU/data center capacity compared to the US/China.
* **Ethical Concerns**: Job displacement, bias in algorithms, and misuse of AI.

---

## 🌈 The Future Vision (Next 10–20 years)

* **Inclusive AI**: Affordable AI-powered healthcare and education for rural India.
* **AI in Democracy**: Smarter governance, transparent systems, AI-driven policymaking.
* **Economic Boost**: AI could add **$1 trillion+ to India’s economy by 2035** (Accenture report).
* **AI Sovereignty**: Development of **indigenous AI models** trained on Indian data, culture, and languages.
* **Human–AI Collaboration**: Focus on AI as a **copilot for humans**, not a replacement.

---

📌 **Takeaway:**
AI has the potential to **redefine India’s growth story**, much like IT did in the 90s. With the right balance of **policy, innovation, and ethics**, India can lead the **AI revolution** not just for itself, but for the **Global South**. 🌍🇮🇳
"""

In [17]:
prompt = f"Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n{essay}"

output = structured_model.invoke(prompt)
output

{'feedback': "The essay is exceptionally well-structured, clearly organized, and presents a comprehensive overview of AI's current landscape and future potential in India. The use of bullet points and emojis enhances readability without compromising professionalism. The content is factually accurate, with relevant statistics (e.g., 5,000+ AI startups, $1 trillion economic impact) and references to key initiatives like Digital India and NITI Aayog. The language is precise, formal, and appropriate for an academic or policy-oriented audience. Each section flows logically into the next, building a compelling narrative from现状 to vision. The conclusion effectively ties together the themes of inclusion, innovation, and global leadership. Minor improvements could include slightly more depth in discussing ethical challenges and expanding on how indigenous AI models might be developed. However, these are minor enhancements — the essay is already outstanding.",
 'score': 10}

In [18]:
print(output["feedback"])
print(output["score"])

The essay is exceptionally well-structured, clearly organized, and presents a comprehensive overview of AI's current landscape and future potential in India. The use of bullet points and emojis enhances readability without compromising professionalism. The content is factually accurate, with relevant statistics (e.g., 5,000+ AI startups, $1 trillion economic impact) and references to key initiatives like Digital India and NITI Aayog. The language is precise, formal, and appropriate for an academic or policy-oriented audience. Each section flows logically into the next, building a compelling narrative from现状 to vision. The conclusion effectively ties together the themes of inclusion, innovation, and global leadership. Minor improvements could include slightly more depth in discussing ethical challenges and expanding on how indigenous AI models might be developed. However, these are minor enhancements — the essay is already outstanding.
10


In [19]:
class UPSCState(TypedDict):

    essay_text: str

    language_feedback: str
    analysis_feedback: str
    clarity_feedback: str

    overall_feedback: str

    individual_scores: Annotated[list[int], operator.add]
    avg_score: float

In [20]:
def evaluate_language(state: UPSCState) -> UPSCState:
    
    prompt = f"Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n{state['essay_text']}"
    output = structured_model.invoke(prompt)
    print(output)

    return {'language_feedback': output["feedback"], 'individual_scores': [output["score"]]}


In [21]:
def evaluate_analysis(state: UPSCState) -> UPSCState:
    
    prompt = f"Evaluate the depth analysis of the following essay and provide a feedback and assign a score out of 10 \n{state['essay_text']}"
    output = structured_model.invoke(prompt)

    return {'analysis_feedback': output["feedback"], 'individual_scores': [output["score"]]}

In [22]:
def evaluate_thought(state: UPSCState) -> UPSCState:

    prompt = f"Evaluate the clarity of thought of the following essay and provide a feedback and assign a score out of 10 \n{state['essay_text']}"
    output = structured_model.invoke(prompt)

    return {'clarity_feedback': output["feedback"], 'individual_scores': [output["score"]]}


In [23]:
def final_evaluation(state: UPSCState) -> UPSCState:
    # Fixed: Use correct state keys
    prompt = f"""Based on the following feedback, create a summarized evaluation:
        
    Language feedback: {state['language_feedback']}
    Analysis feedback: {state['analysis_feedback']} 
    Clarity feedback: {state['clarity_feedback']}

    Provide a comprehensive overall assessment.
    """
    
    overall_feedback = structured_model.invoke(prompt)
    avg_score = sum(state['individual_scores']) / len(state['individual_scores'])
    
    return {'overall_feedback': overall_feedback, 'avg_score': avg_score}


In [24]:
graph = StateGraph(UPSCState)

# nodes
graph.add_node('evaluate_language', evaluate_language)
graph.add_node('evaluate_analysis', evaluate_analysis)
graph.add_node('evaluate_thought', evaluate_thought)

graph.add_node('final_evaluation', final_evaluation)

# edges
graph.add_edge(START, 'evaluate_language')
graph.add_edge(START, 'evaluate_analysis')
graph.add_edge(START, 'evaluate_thought')

graph.add_edge('evaluate_language', 'final_evaluation')
graph.add_edge('evaluate_analysis', 'final_evaluation')
graph.add_edge('evaluate_thought', 'final_evaluation')

graph.add_edge('final_evaluation', END)

workflow = graph.compile()

In [26]:
# workflow
from IPython.display import Image
Image(workflow.get_graph().draw_mermaid_png())

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [27]:
# Exectution
initial_state = {
    'essay_text': essay
}

workflow.invoke(initial_state)

{'feedback': "The essay is exceptionally well-structured, clear, and engaging. It effectively organizes complex information into logical sections with intuitive headings and bullet points, enhancing readability. The language is precise, formal, and appropriate for an academic or policy-oriented audience. Technical terms (e.g., NLP, AGI, precision farming) are used accurately and contextually. The essay demonstrates strong vocabulary, varied sentence structure, and seamless transitions between ideas. The use of emojis is unconventional in formal writing but adds visual appeal without undermining credibility — though in strictly academic contexts, they might be omitted. The conclusion powerfully ties the essay together with a compelling vision, supported by credible references (e.g., Accenture report). Minor improvement: Consider expanding slightly on how 'AI sovereignty' might be practically achieved, or briefly mention public-private collaboration models. Overall, this is a model essay

{'essay_text': '\n# 🌟 AI and Its Future in India\n\n## 📌 Current Landscape\n\n* **Rapid Growth**: India is among the **top 5 countries** in AI research publications.\n* **Government Push**: Initiatives like **Digital India, National AI Strategy (NITI Aayog), and IndiaAI Mission** aim to build AI as a core pillar of development.\n* **Startups & Industry**: Over **5,000+ AI startups** in India, focusing on healthtech, fintech, edtech, and agritech.\n* **Skilled Workforce**: India produces the **largest pool of STEM graduates** annually, making it a hub for AI talent.\n\n---\n\n## 🔑 Key Application Areas\n\n1. **Healthcare 🏥**\n\n   * AI in medical imaging, disease prediction, and telemedicine.\n   * Affordable AI-driven solutions for rural healthcare.\n\n2. **Agriculture 🌾**\n\n   * Precision farming with drones & AI sensors.\n   * Predictive analytics for weather & crop yield.\n   * Reducing supply chain inefficiencies.\n\n3. **Education 🎓**\n\n   * Personalized learning platforms with 